In [11]:
#import beautifulsoup package so we can use it to read html websites. 
# We also have to install the package in your python environment ie easy_install beautifulsoup4, easy_install lxml (subpacksge)
from bs4 import BeautifulSoup
import requests
import pandas as pd

#read from the website
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

#check successful read with a status code of 200
page.status_code

200

In [12]:
# Parse the page with html parser

soup = BeautifulSoup(page.content, 'html.parser')
table = soup.find('table')

# Read the table rows by readding all tr tags
table_rows = table.find_all('tr')
res = []

#iterate through the table rows and add it to a list
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        
        # Check if a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
        if row[2] == 'Not assigned':
         row[2]=row[1]  
        
        #Ignore cells with a borough that is Not assigned.
        if row[1] != 'Not assigned': 
         res.append(row)
        
#load it to a dataframe  
df = pd.DataFrame(res, columns=['PostalCode','Borough','Neightbourhood'])
group= df.groupby(['PostalCode','Borough'])

#More than one neighborhood can exist in one postal code area. Check for them and combine them seperated by comma
df2 = group.apply(lambda x: ','.join(x['Neightbourhood'])).reset_index()
df2.columns = ['PostalCode','Borough','Neightbourhood']
df2.shape


(103, 3)